In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O data/input.txt

--2024-02-25 18:37:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘data/input.txt’

data/input.txt      100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-02-25 18:37:15 (29.3 MB/s) - ‘data/input.txt’ saved [1115394/1115394]



In [6]:
with open('data/input.txt', 'r') as file:
    text = file.read()

In [7]:
print(f"length of dataset in chars: {len(text)}")

length of dataset in chars: 1115394


In [10]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocab size: {vocab_size}")
print(''.join(chars))

vocab size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [14]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode("hello, world"))
print(decode(encode("hello, world")))

[46, 43, 50, 50, 53, 6, 1, 61, 53, 56, 50, 42]
hello, world


In [18]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [20]:
train_size = int(len(data) * 0.9)
train_data = data[:train_size]
val_data = data[train_size:]

In [22]:
context_length = 8
train_data[:context_length+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [31]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
  context = x[:t+1]
  target = y[t]
  print(f"context={context} target={target} ({decode(context.tolist())}->{decode([target.item()])})")


context=tensor([18]) target=47 (F->i)
context=tensor([18, 47]) target=56 (Fi->r)
context=tensor([18, 47, 56]) target=57 (Fir->s)
context=tensor([18, 47, 56, 57]) target=58 (Firs->t)
context=tensor([18, 47, 56, 57, 58]) target=1 (First-> )
context=tensor([18, 47, 56, 57, 58,  1]) target=15 (First ->C)
context=tensor([18, 47, 56, 57, 58,  1, 15]) target=47 (First C->i)
context=tensor([18, 47, 56, 57, 58,  1, 15, 47]) target=58 (First Ci->t)


In [34]:
torch.manual_seed(1337)
batch_size = 4 # number of sequences in a batch
context_length = 8 # length of a sequence

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(0, len(data) - context_length, (batch_size,))
  x = torch.stack([data[idx:idx+context_length] for idx in ix])
  y = torch.stack([data[idx+1:idx+context_length+1] for idx in ix])
  return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("-----")

for batch in range(batch_size):
  for time in range(context_length):
    context = xb[batch, :time+1]
    target = yb[batch, time]
    print(f"context={context} target={target} ({decode(context.tolist())}->{decode([target.item()])})")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
context=tensor([24]) target=43 (L->e)
context=tensor([24, 43]) target=58 (Le->t)
context=tensor([24, 43, 58]) target=5 (Let->')
context=tensor([24, 43, 58,  5]) target=57 (Let'->s)
context=tensor([24, 43, 58,  5, 57]) target=1 (Let's-> )
context=tensor([24, 43, 58,  5, 57,  1]) target=46 (Let's ->h)
context=tensor([24, 43, 58,  5, 57,  1, 46]) target=43 (Let's h->e)
context=tensor([24, 43, 58,  5, 57,  1, 46, 43]) target=39 (Let's he->a)
context=tensor([44]) target=53 (f->o)
context=tensor([44, 53]) target=56 (fo->r)
context=tensor([44, 53, 56]) target=1 (for-> )
context=tensor([44, 53, 56,  1]

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, targets=None):

    logits = self.token_embedding_table(x)

    loss = None
    if targets is not None:
      batch, time, channels = logits.shape
      logits = logits.view(batch*time, channels)
      targets = targets.view(batch * time)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, x, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(x)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      next_token = torch.multinomial(probs, 1)
      x = torch.cat([x, next_token.view(-1, 1)], dim=1)
    return x


In [81]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.4194, grad_fn=<NllLossBackward0>)


In [82]:
x = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(x, 100)[0].tolist()))


EJEINVSj,$HV 3
hQULWs,&xrzVifhLsGucg?&oZhBzvQXu,fqdhA!XOJPpNBKgoE
 IMn Y$qTMyuPqL.kQ'G& HCTUqH'$zUSm


In [83]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [123]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  # if steps % 10 == 0:
  #   print(f"step={steps} loss={loss.item()}")
print(f"loss={loss.item()}")

loss=2.447760820388794


In [ ]:
x = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(x, max_new_tokens=500)[0].tolist()))


pot mbr I's frain lasep f cow; hakn.

OUT:

MELeigo p,
M:


I sh hit mpunt n,
Thule.
Thand k? yondr,
Te?
tr thisopth t:
O:

I wig nt et ha faghir lleneew; t brs E wivet lin thed g atiroular ' bulime,
Hat
m os ive be;
Fir is
Maroof tharountrius, n ly ms Whagend athins,

Aned alllinamyoumadebldivean the
BIIt bel l h gured ipe: le thisther ly ak hico ig s d rairin;
Hexthe y meay wosimo I thies t cor coon ucapr, h. oung.
He ce the.
URD:
ABORWhot blunougherra th wid g y y balor cy oumor. berisedoad;



# The math trick in self-attention

In [4]:
import torch
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


In [6]:
# x[b, t] = mean of x[b, i] for i in 0 to t
xbow = torch.zeros((B, T, C)) # bow = bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, dim=0)

In [18]:
# doing matmul with a lower triangular matrix is equivalent to the above lol genius
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)
xbow2 = weights @ x
torch.allclose(xbow, xbow2)

True

In [20]:
# can also use F.softmax to get the weights because the lower triangular matrix is just a special case of a weight matrix
import torch.nn.functional as F
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril ==0 , float('-inf'))
weights = F.softmax(weights, dim=1)
xbow3 = weights @ x
torch.allclose(xbow, xbow3)

True